<a href="https://colab.research.google.com/github/martinmwarangu/Pyspark_Msc/blob/main/CN7030_week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e8a87e3b04718a4b719415afa0770e8b8b214c7a2b045e1990a26c4072d50e03
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
ds = spark.read.csv("/content/drive/MyDrive/Data/bezdekIris.data", inferSchema=True,header=True)\
.toDF("sep_len", "sep_wid", "pet_len", "pet_wid", "label")
ds.select("label").distinct().show(10)
ds.count()

+---------------+
|          label|
+---------------+
| Iris-virginica|
|    Iris-setosa|
|Iris-versicolor|
+---------------+



149

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler(\
inputCols=["sep_len", "sep_wid", "pet_len", "pet_wid"],\
outputCol="features")
df_temp = vector_assembler.transform(ds)
df_temp.show(5)

+-------+-------+-------+-------+-----------+-----------------+
|sep_len|sep_wid|pet_len|pet_wid|      label|         features|
+-------+-------+-------+-------+-----------+-----------------+
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|    5.4|    3.9|    1.7|    0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
+-------+-------+-------+-------+-----------+-----------------+
only showing top 5 rows



In [ ]:
# drop unecessary columns
df = df_temp.drop('sep_len', 'sep_wid', 'pet_len', 'pet_wid')
df.show(5)

+-----------+-----------------+
|      label|         features|
+-----------+-----------------+
|Iris-setosa|[4.9,3.0,1.4,0.2]|
|Iris-setosa|[4.7,3.2,1.3,0.2]|
|Iris-setosa|[4.6,3.1,1.5,0.2]|
|Iris-setosa|[5.0,3.6,1.4,0.2]|
|Iris-setosa|[5.4,3.9,1.7,0.4]|
+-----------+-----------------+
only showing top 5 rows



In [ ]:
#index the labels
from pyspark.ml.feature import StringIndexer
l_indexer = StringIndexer(inputCol="label", outputCol="labelIndex")
df = l_indexer.fit(df).transform(df)

In [ ]:
df.select('label','labelIndex').distinct().show(3)

+---------------+----------+
|          label|labelIndex|
+---------------+----------+
|Iris-versicolor|       0.0|
| Iris-virginica|       1.0|
|    Iris-setosa|       2.0|
+---------------+----------+



In [ ]:
#splitting training and testing data 70-30
(trainingData , testData) = df.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
dt = DecisionTreeClassifier(labelCol="labelIndex",featuresCol="features",impurity="entropy",maxDepth= 4,seed=1234)
model = dt.fit(trainingData)
predictions = model.transform(testData)

In [ ]:
# checking accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex",predictionCol="prediction",\
                                             metricName="accuracy" ,)
accuracy = evaluator.evaluate(predictions)
print("Test accuracy = " ,accuracy)
print(model.toDebugString)



Test accuracy =  0.8947368421052632
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_a83b02e992b0, depth=4, numNodes=11, numClasses=3, numFeatures=4
  If (feature 2 <= 2.5999999999999996)
   Predict: 2.0
  Else (feature 2 > 2.5999999999999996)
   If (feature 2 <= 4.75)
    If (feature 3 <= 1.55)
     Predict: 0.0
    Else (feature 3 > 1.55)
     Predict: 1.0
   Else (feature 2 > 4.75)
    If (feature 3 <= 1.75)
     If (feature 1 <= 2.6500000000000004)
      Predict: 1.0
     Else (feature 1 > 2.6500000000000004)
      Predict: 0.0
    Else (feature 3 > 1.75)
     Predict: 1.0



In [ ]:
# multiple classification using logistic regression
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LogisticRegression
(train,test) = df.randomSplit([0.7,0.3],seed=2018)
lr = LogisticRegression(maxIter=100,\
                        featuresCol="features",\
                        labelCol="labelIndex",)
ovr = OneVsRest(classifier= lr,\
                labelCol="labelIndex",\
                featuresCol="features")
ovrModel = ovr.fit(train)
predictionsovr = ovrModel.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="labelIndex", predictionCol="prediction",\
metricName="accuracy")
accuracy = evaluator.evaluate(predictionsovr)
print("Test accuracy =  " , accuracy)

Test accuracy =   0.9361702127659575


In [ ]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="labelIndex", predictionCol="prediction",\
metricName="weightedPrecision")
accuracy = evaluator.evaluate(predictionsovr)
print("precision  =  " , accuracy)

precision  =   0.9448742746615086


In [ ]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="labelIndex", predictionCol="prediction",\
metricName="weightedRecall")
accuracy = evaluator.evaluate(predictionsovr)
print("Recall =  " , accuracy)

Recall =   0.9361702127659575
